In [1]:
using ColoringNames


INFO: Recompiling stale cache file /home/wheel/oxinabox/.julia/lib/v0.6/ColoringNames.ji for module ColoringNames.


LoadError: LoadError: [91mDeclaring __precompile__(true) is only allowed in module files being imported.[39m
while loading /home/wheel/oxinabox/.julia/v0.6/ColoringNames/src/ColoringNames.jl, in expression starting on line 1

In [2]:
cldata = load_munroe_data(dev_as_train=false, dev_as_test=true)

LoadError: [91mUndefVarError: load_munroe_data not defined[39m

In [3]:
function qualitative_demo(mdl, do_oov=true)
    names = ["brownish green", "brown", "brownish", "greenish", "greenish brown", "green"]
    if do_oov
        append!(names, ["ish"])
    end
    
    cols = mapslices(hsv2colorant, (query(mdl, names)), 1)
    display(cols)
    println(join(rpad.(names, 10), "|"))
end

qualitative_demo(mdl::TermToColorPointEmpirical) = qualitative_demo(mdl, false)

LoadError: [91mUndefVarError: TermToColorPointEmpirical not defined[39m

In [4]:
noml = ColoringNames.TermToColorPointEmpirical()
train!(noml, cldata)

qualitative_demo(noml)
@show evaluate(noml, cldata.dev)

LoadError: [91mUndefVarError: ColoringNames not defined[39m

In [5]:
sowe = TermToColorPointSOWE(cldata.encoding; n_steps=4)

#train!(noml, cldata)
query(sowe, ["red", "blue"])
qualitative_demo(sowe)
#@show evaluate(sowe, cldata.dev)

LoadError: [91mUndefVarError: TermToColorPointSOWE not defined[39m

In [87]:
cldata.train.colors


1523108×3 Array{Float32,2}:
 0.344113  0.855319  0.921569
 0.281905  0.744681  0.921569
 0.347551  0.921397  0.898039
 0.311209  0.900398  0.984314
 0.312085  0.992095  0.992157
 0.22766   0.987395  0.933333
 0.288462  0.466368  0.87451 
 0.276936  0.804878  0.964706
 0.325359  0.95      0.862745
 0.385201  0.971774  0.972549
 0.212885  0.955823  0.976471
 0.346405  0.6       1.0     
 0.238397  0.42246   0.733333
 ⋮                           
 0.1621    0.421965  0.678431
 0.153333  0.722543  0.678431
 0.201754  0.256757  0.580392
 0.264463  0.620513  0.764706
 0.144608  0.790698  0.67451 
 0.178125  0.97561   0.643137
 0.166667  0.350962  0.815686
 0.174603  0.640244  0.643137
 0.587607  0.397959  0.768627
 0.774242  0.973451  0.443137
 0.376894  0.768559  0.898039
 0.179104  0.41358   0.635294

In [66]:
typeof(sowe)

ColoringNames.TermToColorPointSOWE{TensorFlow.Tensor{Any},MLLabelUtils.LabelEnc.NativeLabels{SubString{String},307},TensorFlow.Tensor{String}}

In [56]:
TensorFlow.Ops.atan2 |> methods

# 25 methods for generic function "atan2":
atan2(y::Float64, x::Float64) in Base.Math at math.jl:497
atan2(y::Float32, x::Float32) in Base.Math at math.jl:498
atan2(a::Float16, b::Float16) in Base.Math at math.jl:957
atan2(y::BigFloat, x::BigFloat) in Base.MPFR at mpfr.jl:661
atan2(y::T, x::T) where T<:AbstractFloat in Base.Math at math.jl:495
atan2(x::ForwardDiff.Dual{Txy,V,N} where N where V<:Real, y::ForwardDiff.Dual{Txy,V,N} where N where V<:Real) where Txy in ForwardDiff at /home/wheel/oxinabox/.julia/v0.6/ForwardDiff/src/dual.jl:99
atan2(x::ForwardDiff.Dual{Tx,V,N} where N where V<:Real, y::ForwardDiff.Dual{Ty,V,N} where N where V<:Real) where {Tx, Ty} in ForwardDiff at /home/wheel/oxinabox/.julia/v0.6/ForwardDiff/src/dual.jl:100
atan2(x::ForwardDiff.Dual{Tx,V,N} where N where V<:Real, y::AbstractFloat) where Tx in ForwardDiff at /home/wheel/oxinabox/.julia/v0.6/ForwardDiff/src/dual.jl:104
atan2(x::AbstractFloat, y::ForwardDiff.Dual{Ty,V,N} where N where V<:Real) where Ty in ForwardDiff at /home/wheel/oxinabox/.julia/v0.6/ForwardDiff/src/dual.jl:105
atan2(x::ForwardDiff.Dual{Tx,V,N} where N where V<:Real, y::Irrational) where Tx in ForwardDiff at /home/wheel/oxinabox/.julia/v0.6/ForwardDiff/src/dual.jl:104
atan2(x::Irrational, y::ForwardDiff.Dual{Ty,V,N} where N where V<:Real) where Ty in ForwardDiff at /home/wheel/oxinabox/.julia/v0.6/ForwardDiff/src/dual.jl:105
atan2(x::ForwardDiff.Dual{Tx,V,N} where N where V<:Real, y::Integer) where Tx in ForwardDiff at /home/wheel/oxinabox/.julia/v0.6/ForwardDiff/src/dual.jl:104
atan2(x::Integer, y::ForwardDiff.Dual{Ty,V,N} where N where V<:Real) where Ty in ForwardDiff at /home/wheel/oxinabox/.julia/v0.6/ForwardDiff/src/dual.jl:105
atan2(x::ForwardDiff.Dual{Tx,V,N} where N where V<:Real, y::Rational) where Tx in ForwardDiff at /home/wheel/oxinabox/.julia/v0.6/ForwardDiff/src/dual.jl:104
atan2(x::Rational, y::ForwardDiff.Dual{Ty,V,N} where N where V<:Real) where Ty in ForwardDiff at /home/wheel/oxinabox/.julia/v0.6/ForwardDiff/src/dual.jl:105
atan2(x::ForwardDiff.Dual{Tx,V,N} where N where V<:Real, y::Real) where Tx in ForwardDiff at /home/wheel/oxinabox/.julia/v0.6/ForwardDiff/src/dual.jl:104
atan2(x::Real, y::ForwardDiff.Dual{Ty,V,N} where N where V<:Real) where Ty in ForwardDiff at /home/wheel/oxinabox/.julia/v0.6/ForwardDiff/src/dual.jl:105
atan2(y::Real, x::Real) in Base.Math at math.jl:494
atan2(x::Number, y::AbstractArray{T1,N} where N) where T1<:Number in Base at deprecated.jl:56
atan2(x::AbstractArray{T1,N} where N, y::Number) where T1<:Number in Base at deprecated.jl:56
atan2(x::AbstractArray{T1,N} where N, y::AbstractArray{T2,N} where N) where {T1<:Number, T2<:Number} in Base at deprecated.jl:56
atan2(x::ForwardDiff.Dual{Tx,V,N} where N where V<:Real, y::RoundingMode) where Tx in ForwardDiff at /home/wheel/oxinabox/.julia/v0.6/ForwardDiff/src/dual.jl:104
atan2(x::RoundingMode, y::ForwardDiff.Dual{Ty,V,N} where N where V<:Real) where Ty in ForwardDiff at /home/wheel/oxinabox/.julia/v0.6/ForwardDiff/src/dual.jl:105
atan2(y::TensorFlow.Tensor{T1}, x::TensorFlow.Tensor{T2}) where {T1, T2} in ColoringNames at /home/wheel/oxinabox/.julia/v0.6/ColoringNames/src/tf_extra.jl:3
atan2(y::TensorFlow.Tensor{T1}, x::TensorFlow.Tensor{T2}, ϵ) where {T1, T2} in ColoringNames at /home/wheel/oxinabox/.julia/v0.6/ColoringNames/src/tf_extra.jl:3